In [ ]:
# Imports
from openai import OpenAI, APIError, OpenAIError
import requests
from bs4 import BeautifulSoup
from langdetect import detect
from pydantic import BaseModel, ValidationError
import datetime
import logging
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type, before_sleep_log
import your_information

In [ ]:
# Configure logging 
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Web Scraping 
Extract job description text directly from the job posting URL using `Requests` and `BeautifulSoup`.

In [ ]:
# HTTP request headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}


@retry(
    retry=retry_if_exception_type((
        requests.exceptions.ConnectionError,
        requests.exceptions.Timeout,
        requests.exceptions.HTTPError
    )),
    stop=stop_after_attempt(3),
    wait=wait_exponential(min=4, max=10),  # Wait 4-10 sec between retries
    before_sleep=before_sleep_log(logging.getLogger(), logging.INFO)
)
def get_html_content(url):
    # Send HTTP GET request
    url_response = requests.get(url, headers=headers, timeout=10)
    url_response.raise_for_status()  
    # Return HTML content
    return url_response.text


try:
    # Get HTML content
    html_content = get_html_content(your_information.job_posting_url)
    
    # Initialize BeautifulSoup object to parse HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Remove unwanted HTML elements
    for element in soup(["head", "header", "footer", "script", "style", "nav", "form", "aside", "iframe"]):
        element.decompose()
    
    # Extract the text
    job_description = soup.get_text()
    
    # Remove whitespace characters
    job_description = " ".join(job_description.split())  
    
    # Log job description text as info
    logging.info(f"Successfully retrieved the following job description text:\n\n{job_description}")

except requests.exceptions.RequestException as e:
    logging.error(f"Request failed: {e}")
    raise
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
    raise

Automatically detect the language of the job description using `langdetect`.

In [ ]:
# Detect language of job description 
# Note: Returns ISO 639-1 language codes (e.g., 'en' for English or 'de' for German)
try:
    language = detect(job_description)

    # Log detected language as info if it's English or German
    if language in ["en", "de"]:
        logging.info(f"Detected language of job description: {language}")
    else:
        # Log an error if the language is not English or German
        logging.error(f"Unsupported language of job description detected: {language}")
        raise

except Exception as e:
    # Log an error if language detection fails
    logging.error(f"Language detection failed: {e}")
    raise

# Information Extraction 
Use `OpenAI's chat completions API (beta)` with `structured outputs` to extract the relevant information from the job description in JSON format. Leverage `Pydantic` to validate the JSON format against a defined schema.

In [ ]:
# Initialize OpenAI client to connect to ChatGPT API
client = OpenAI(api_key=your_information.OPENAI_API_KEY)

# Define JSON output format
class JobDescriptionExtraction(BaseModel):
    employer: str
    job_title: str
    requirements: list[str]
    tasks: list[str]
    contact_person: str
    address: str

# Define system prompt for information extraction in English and German
extraction_system_prompt = {
    "en": """
    You are an expert at structured data extraction. You will be given unstructured text from a job description 
    and should convert it into the given structure. If the information is missing in the job description, use 
    "unknown" as the value. Respond as concisely as possible.
    """,
    "de": """
    Du bist ein Experte für die Extraktion strukturierter Daten. Dir wird unstrukturierter Text aus einer 
    Stellenbeschreibung gegeben, den du in das vorgegebene Format umwandeln sollst. Wenn die Information 
    in der Stellenbeschreibung fehlt, verwende "unbekannt" als Wert. Antworte so kurz wie möglich.
    """
}

# Extract information from job description text 
try:
    # Get the raw information extraction response (Note: This will make 1 API request to the chat completions beta endpoint)
    extraction_response_raw = client.beta.chat.completions.parse(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": extraction_system_prompt[language]},  
            {"role": "user", "content": job_description}
        ],
        response_format = JobDescriptionExtraction 
    )

    # Get the parsed information extraction response and convert it to a dict
    job_information = extraction_response_raw.choices[0].message.parsed.dict()

    # Log extracted job information as info
    logging.info(f"Successfully retrieved the following job information dictionary:\n\n{job_information}")

except KeyError as e:
    logging.error(f"Dictionary key error: {e}")
    raise
except APIError as e:
    logging.error(f"OpenAI API request error: {e}")
    raise
except ValidationError as e:
    logging.error(f"Data validation error: {e}")
    raise
except OpenAIError as e:
    logging.error(f"General OpenAI error: {e}")
    raise
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")
    raise

# Cover Letter Generation  
Produce three unique cover letter suggestions by aligning your education, work experience, skills, and motivation to the tasks and requirements outlined in the job description using `OpenAI's chat completions API` and language-specific prompts in both English and German.

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, system_prompt="", model="gpt-4o-mini", temperature=0.7): 
    try:
        response = client.chat.completions.create(
            model = model,
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature = temperature  # controls randomness of generated text with values from 0 (little to no randomness) to 1 (maximum randomness)
        )
        return response
    
    except APIError as e:
        logging.error(f"OpenAI API request error: {e}")
        raise
    except OpenAIError as e:
        logging.error(f"General OpenAI error: {e}")
        raise
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        raise
        
# System prompt for cover letter generation in English and German
cover_letter_system_prompt = {
    "en": """
    You are an expert assistant in creating professional cover letters directed towards hiring managers
    with the following capabilities:
    
    STYLE AND TONE:
    - Write in a confident, professional tone that avoids clichés and generic phrases
    - Use active voice and concrete examples
    - Keep sentences concise and well-structured
    - Maintain formal business language while showing personality
    
    CONTENT STRUCTURE:
    - Create a clear three-part structure: strong opening, relevant middle section, confident closing
    - Focus on matching candidate's qualifications to job requirements
    - Highlight unique selling points and specific achievements
    - Avoid repeating CV/resume content verbatim
    
    FORMATTING GUIDELINES:
    - Provide response in plain text without any Markdown formatting
    - Follow standard business letter formatting
    - Limit to one page (approximately 400 words maximum)
    - Use appropriate salutations and closings based on country conventions
    
    CUSTOMIZATION APPROACH:
    - Adapt tone and content to industry standards and company culture
    - Include specific references to the company and position
    - Emphasize relevant experiences and skills for the role
    - Connect candidate's motivation to company's mission/values
    
    IMPORTANT RULES:
    - Never include false or exaggerated information
    - Avoid generic templates and obvious mass-application language
    - Don't apologize for missing qualifications; focus on growth potential
    - Maintain professionalism while showing genuine enthusiasm
    """,
    
    "de": """
    Du bist ein Experte im Erstellen professioneller Bewerbungsanschreiben, die an Personalverantwortliche 
    gerichtet sind, mit folgenden Fähigkeiten:
    
    STIL UND TON:
    - Schreibe in einem selbstbewussten, professionellen Ton ohne Floskeln und ohne Standardphrasen
    - Verwende aktive Formulierungen und konkrete Beispiele
    - Halte Sätze prägnant und gut strukturiert
    - Bewahre formelle Geschäftssprache und zeige gleichzeitig Persönlichkeit
    
    INHALTLICHE STRUKTUR:
    - Erstelle eine klare, dreiteilige Struktur: starke Einleitung, relevanter Hauptteil, selbstbewusster Abschluss
    - Konzentriere dich auf die Übereinstimmung der Qualifikationen mit den Jobanforderungen
    - Hebe Alleinstellungsmerkmale und spezifische Erfolge hervor
    - Vermeide wörtliche Wiederholungen aus dem Lebenslauf
    
    FORMATIERUNGSRICHTLINIEN:
    - Gib die Antwort als reinen Text ohne Markdown-Formatierung aus
    - Verwende das Standardformat für Geschäftsbriefe
    - Beschränke die Länge auf eine Seite (maximal circa 400 Wörter)
    - Verwende angemessene Anreden und Grußformeln nach deutschen Konventionen
    
    ANPASSUNGSSTRATEGIE:
    - Passe Ton und Inhalt an Branchenstandards und Unternehmenskultur an
    - Beziehe dich konkret auf das Unternehmen und die Position
    - Betone relevante Erfahrungen und Fähigkeiten für die Rolle
    - Verbinde die Motivation des Bewerbers mit der Mission bzw. den Werten des Unternehmens
    
    WICHTIGE REGELN:
    - Keine falschen oder übertriebenen Angaben machen
    - Vermeide allgemeine Formulierungen, die nach standardisierten Massenanschreiben klingen
    - Entschuldige dich nicht für fehlende Qualifikationen; fokussiere auf Entwicklungspotenzial
    - Bewahre Professionalität und zeige gleichzeitig authentische Begeisterung
    """
}

# Cover letter prompt in English and German
cover_letter_prompt = {
    "en": f"""
    Your task is to create a professional cover letter.
    
    Address the letter to the following employer, address, job position, and contact person:
    Employer: {job_information["employer"]}
    Address: {job_information["address"]}
    Job Position: {job_information["job_title"]}
    Contact Person: {job_information["contact_person"]}
    
    Use the following sender information:
    Name: {your_information.name}
    Address: {your_information.address}
    Phone Number: {your_information.phone}
    Email: {your_information.email}
    
    Include the location and date in the letterhead of the cover letter.
    Use the location from: {your_information.address}
    Use the current date in the English date format: {datetime.date.today()}
    
    Describe how the education, work experience, skills, and motivation fulfill 
    the job requirements and tasks. Use the following information:
    Requirements: {job_information["requirements"]}
    Tasks: {job_information["tasks"]}
    Education: {your_information.education}
    Work Experience: {your_information.work_experience}
    Skills: {your_information.skills}
    Motivation: {your_information.motivation}
    
    State the salary expectations and possible start date as follows:
    Salary Expectations: {your_information.salary_expectations}
    Possible Start Date: {your_information.possible_start_date}
    
    Write in a professional, concise, and compact tone.
    
    Sign the cover letter as {your_information.name}.
    """,
    
    "de": f"""
    Deine Aufgabe ist es, ein professionelles Bewerbungsanschreiben zu erstellen.
    
    Adressiere das Anschreiben an folgenden Arbeitgeber, Adresse, Stelle und Kontaktperson:
    Arbeitgeber: {job_information["employer"]}
    Adresse: {job_information["address"]}
    Stelle: {job_information["job_title"]}
    Kontaktperson: {job_information["contact_person"]}
    
    Verwende folgenden Absender:
    Name: {your_information.name}
    Adresse: {your_information.address}
    Telefonnummer: {your_information.phone}
    E-Mail: {your_information.email}
    
    Verwende im Briefkopf des Anschreibens Ort und Datum. 
    Verwende den Ort aus: {your_information.address}
    Verwende das folgende, aktuelle Datum im deutschen Datumsformat: {datetime.date.today()}
    
    Beschreibe inwiefern die Anforderungen und Aufgaben erfüllt werden durch die 
    Ausbildung, Arbeitserfahrung, Kompetenzen und Motivation. 
    Verwende hierfür die folgenden Informationen:
    Anforderungen: {job_information["requirements"]}
    Aufgaben: {job_information["tasks"]}
    Ausbildung: {your_information.education}
    Arbeitserfahrung: {your_information.work_experience}
    Kompetenzen: {your_information.skills}
    Motivation: {your_information.motivation}
    
    Nenne folgende Gehaltsvorstellung und mögliches Eintrittsdatum:
    Gehaltsvorstellung : {your_information.salary_expectations}
    Mögliches Eintrittsdatum: {your_information.possible_start_date}
    
    Schreibe in einem professionellen, präzisen und kompakten Ton.
    
    Unterschreibe das Anschreiben als {your_information.name}.
    """
}

# Create 3 cover letter suggestions (Note: This will make 3 API requests to the chat completions endpoint)
cover_letters = []
for i in range(3):
    # Generate a single cover letter
    cover_letter = get_completion(
        prompt=cover_letter_prompt[language],  
        system_prompt=cover_letter_system_prompt[language]
    ).choices[0].message.content 
    # Append cover letter to list
    cover_letters.append(cover_letter)

# Log cover letter suggestions as info
for index, cover_letter in enumerate(cover_letters, start=1):
    message = (
        f"{'Anschreiben' if language=='de' else 'Cover Letter'} {index}\n"
        f"{'=' * 50}\n"
        f"{cover_letter}\n"
        f"{'=' * 50}"
    )
    logging.info(message)

# Cover Letter Refinement  
Reviews and refines each cover letter leveraging `chat completions API` with specific refinement prompts. 

In [ ]:
# System prompt for cover letter refinement in English and German
refinement_system_prompt = {
    "en": """
    You are an expert cover letter reviewer and editor with extensive professional recruitment experience. 
    Your task is to enhance cover letters with a focus on:

    CRITERIA:
    - Impact and Persuasiveness: Ensure the letter convincingly demonstrates value
    - Relevance and Specificity: Confirm qualifications match job requirements with concrete examples
    - Flow and Tone: Ensure logical structure, smooth transitions, and a professional tone

    GUIDELINES:
    - Strengthen key value propositions and remove clichés
    - Emphasize relevant achievements and unique qualifications
    - Improve sentence structure and wording
    - Maintain original length, formatting, and contact details
    
    Only return the improved cover letter.
    """,
    
    "de": """
    Du bist ein erfahrener Experte für Bewerbungsanschreiben mit umfangreicher Erfahrung im Recruiting.
    Deine Aufgabe ist es, Anschreiben basierend auf folgenden Kriterien zu verbessern:

    KRITERIEN:
    - Wirkung und Überzeugungskraft: Stärkung der Überzeugungskraft des Anschreibens
    - Relevanz und Konkretheit: Sicherstellung passender Qualifikationen mit konkreten Beispielen
    - Wortfluss und Ton: Achte auf eine logische Struktur, reibungslose Übergänge und einen professionellen Ton

    RICHTLINIEN:
    - Stärkung der wichtigsten Nutzenversprechen und Vermeidung von Standardfloskeln
    - Relevante Erfolge und Alleinstellungsmerkmale betonen
    - Satzstruktur und Wortwahl optimieren
    - Originallänge, Formatierung und Kontaktdaten beibehalten
    
    Gib nur das verbesserte Anschreiben zurück.
    """
}

# Cover letter refinement prompt in English and German
refinement_prompt = {
    "en": """
    Please improve the following cover letter for the position of {job_title}, ensuring it meets these requirements:
    - Emphasizes the following key requirements: {requirements}
    - Addresses main tasks for the role: {tasks}

    Original Cover Letter:
    {cover_letter}
    """,
    
    "de": """
    Bitte verbessere das folgende Anschreiben für die Position als {job_title} und stelle sicher, dass es diesen Anforderungen entspricht:
    - Betonung der folgenden Schlüsselkriterien: {requirements}
    - Bezugnahme auf die Hauptaufgaben der Position: {tasks}

    Ursprüngliches Anschreiben:
    {cover_letter}
    """
}

# Refine cover letters (Note: This will make 3 API requests to the chat completions endpoint)
refined_cover_letters = []
for cover_letter in cover_letters:
    # Create the refinement prompt for the current cover letter
    current_refinement_prompt = refinement_prompt[language].format(
        job_title=job_information["job_title"],
        requirements=job_information["requirements"],
        tasks=job_information["tasks"],
        cover_letter=cover_letter
    )
    # Refine the current cover letter
    refined_cover_letter = get_completion(
        prompt=current_refinement_prompt,
        system_prompt=refinement_system_prompt[language],
        temperature=0.4  # Lower temperature for more consistent refinements
    ).choices[0].message.content
    # Append refined cover letter to list
    refined_cover_letters.append(refined_cover_letter)

# Log refined cover letter suggestions as info
for index, cover_letter in enumerate(refined_cover_letters, start=1):
    message = (
        f"{'Verfeinertes Anschreiben' if language=='de' else 'Refined Cover Letter'} {index}\n"
        f"{'=' * 50}\n"
        f"{cover_letter}\n"
        f"{'=' * 50}"
    )
    logging.info(message)

# Cover Letter Consolidation  
Combines the best parts of each refined cover letter into a single, cohesive cover letter using `chat completions API` with dedicated consolidation prompts.

In [ ]:
# System prompt for cover letter consolidation in English and German 
consolidation_system_prompt = {
    "en": """
    You are an expert cover letter writer and career consultant. Your goal is to create a cohesive, persuasive, and highly professional cover letter 
    by synthesizing the strongest points from multiple drafts. Use impactful language and ensure the letter addresses key job requirements, 
    highlights relevant skills, and maintains a smooth and logical flow. The tone should be professional and engaging, and the final output should 
    be concise yet compelling.
    """,
    
    "de": """
    Du bist ein erfahrener Anschreiben-Experte und Karrierecoach. Dein Ziel ist es, ein zusammenhängendes, überzeugendes und hochprofessionelles 
    Anschreiben zu erstellen, indem du die stärksten Elemente aus mehreren Entwürfen kombinierst. Verwende wirkungsvolle Formulierungen und stelle 
    sicher, dass das Anschreiben die wichtigsten Jobanforderungen anspricht, relevante Fähigkeiten hervorhebt und einen fließenden und logischen 
    Aufbau hat. Der Ton sollte professionell und ansprechend sein, und das Endergebnis sollte prägnant und überzeugend sein.
    """
}


# Cover letter consolidation prompt in English and German 
consolidation_prompt = {
    "en": f"""
    Based on the following three cover letter drafts, please create the best possible cover letter for the position of {job_information["job_title"]}. 

    - Highlight the key requirements: {job_information["requirements"]}
    - Address the main tasks: {job_information["tasks"]}
    - Draw from the strongest phrases, examples, and ideas in each draft to create a cohesive, persuasive, and professional cover letter.
    
    Drafts:
    1. {refined_cover_letters[0]}
    2. {refined_cover_letters[1]}
    3. {refined_cover_letters[2]}

    Return only the consolidated cover letter.
    """,
    
    "de": f"""
    Erstelle auf Basis der folgenden drei Entwürfe das bestmögliche Anschreiben für die Position als {job_information["job_title"]}.
    
    - Hebe die wichtigsten Anforderungen hervor: {job_information["requirements"]}
    - Gehe auf die Hauptaufgaben der Position ein: {job_information["tasks"]}
    - Nutze die besten Formulierungen, Beispiele und Ideen aus jedem Entwurf, um ein zusammenhängendes, überzeugendes und professionelles 
    Anschreiben zu erstellen.
    
    Entwürfe:
    1. {refined_cover_letters[0]}
    2. {refined_cover_letters[1]}
    3. {refined_cover_letters[2]}
    
    Gib nur das konsolidierte Anschreiben zurück.
    """
}

# Consolidate the refined cover letters (Note: This will make 1 API request to the chat completions endpoint)
consolidated_cover_letter = get_completion(
    prompt=consolidation_prompt[language],
    system_prompt=consolidation_system_prompt[language],
    temperature=0.4  
).choices[0].message.content

# Log consolidated cover letter as info
logging.info(f"{'Konsolidiertes Anschreiben' if language=='de' else 'Consolidated Cover Letter'}:\n\n{consolidated_cover_letter}")

# Save Cover Letters as Text File 
Exports the consolidated cover letter and the three refined cover letters to a `.txt` file for easy access and final editing.

In [ ]:
# Save cover letters
try:
    with open("cover_letters/cover_letters.txt", "w", encoding="utf-8") as file:
        # Save consolidated cover letter with formatted header
        header = (
            f"# {'-' * 40}\n"
            f"# {'Konsolidiertes Anschreiben' if language=='de' else 'Consolidated Cover Letter'}\n"
            f"# {'-' * 40}\n\n"
        )
        file.write(header + consolidated_cover_letter + "\n\f\n")  # "\f" separates each cover letter with a page break
        
        # Save all refined cover letters
        for index, cover_letter in enumerate(refined_cover_letters, start=1):
            header = (
                f"# {'-' * 40}\n"
                f"# {'Verfeinertes Anschreiben' if language=='de' else 'Refined Cover Letter'} {index}\n"
                f"# {'-' * 40}\n\n"
            )
            file.write(header + cover_letter + "\n\f\n") 
except Exception as e:
    logging.error(f"Error saving cover letters to .txt file: {e}")
    raise